In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy import interpolate

In [2]:
def createInputFileDictionary(path):
    inputFile = open(path,'r')
    
    inputFileDict = {}
    
    inputLines = inputFile.readlines()
    
    variableValue = ''
    variableName = ''

    sectionName = ''

    for i in range(len(inputLines)):
        line = inputLines[i].strip()
        if len(line) == 0:
            continue
        if line[0] == '!' or line[1] == '!':
            continue
        if line[0] == '&':# and line[1:]!='end':
            #if this is not the first section, add the last variable to the dictionary before moving onto this new section
            if sectionName != '':
                addVariable(inputFileDict, sectionName, variableName, variableValue)
                variableValue = ''
                variableName = ''

            sectionName = line[1:]
            inputFileDict[sectionName] = {}
            continue 


        #"""       
        splitLine = line.split('=')
        
        #if this line is a continuation of a previous variable
        if len(splitLine) == 1:
            splitLine = re.sub("\s+", ",", splitLine[0].strip()).replace(' ',',')
            variableValue = variableValue + splitLine + ','
            if i == len(inputLines) - 1:
                addVariable(inputFileDict, sectionName, variableName, variableValue)
        #if this is a new variable
        else:
            if variableName != '':
                addVariable(inputFileDict, sectionName, variableName, variableValue)
                variableValue = ''
                variableName = ''
            
            variableName = splitLine[0].strip(); variableValue = re.sub("\s+", ",", splitLine[1].strip()).replace(' ',',') + ','
        #"""
            
    return inputFileDict
    
def addVariable(dictionary, sectionName, variableName, variableValue):
    #get rid of extra comma at the end
    #print(f"{sectionName, variableName, variableValue}")
    variableValue = variableValue.replace(',,',',')
    variableValue = variableValue[:-1]
    splitVariable = np.array(variableValue.split(','))
        
    try:  
        if len(splitVariable) > 1:
            dictionary[sectionName][variableName]=np.array([float(var) for var in splitVariable])

        else:
            try: 
                dictionary[sectionName][variableName]=float(variableValue)
            except:
                dictionary[sectionName][variableName]=variableValue
    except Exception as e:
        pass

In [5]:
shot = '174658'
genray_in_path = f'genray.in'
genray_in_dict = createInputFileDictionary(genray_in_path)
density_profile = genray_in_dict['dentab']['prof']
temp_profile = genray_in_dict['temtab']['prof']

print(density_profile)

[4.48613453e+19 4.48613453e+19 4.48613453e+19 4.46052391e+19
 4.46052391e+19 4.46052391e+19 4.39110362e+19 4.39110362e+19
 4.39110362e+19 4.27147693e+19 4.27147693e+19 4.27147693e+19
 4.10693018e+19 4.10693018e+19 4.10693018e+19 3.95449586e+19
 3.95449586e+19 3.95449586e+19 3.81727751e+19 3.81727751e+19
 3.81727751e+19 3.69408239e+19 3.69408239e+19 3.69408239e+19
 3.58330029e+19 3.58330029e+19 3.58330029e+19 3.48182826e+19
 3.48182826e+19 3.48182826e+19 3.38667281e+19 3.38667281e+19
 3.38667281e+19 3.29523368e+19 3.29523368e+19 3.29523368e+19
 3.20500938e+19 3.20500938e+19 3.20500938e+19 3.11325085e+19
 3.11325085e+19 3.11325085e+19 3.01717716e+19 3.01717716e+19
 3.01717716e+19 2.91387608e+19 2.91387608e+19 2.91387608e+19
 2.80132527e+19 2.80132527e+19 2.80132527e+19 2.67948547e+19
 2.67948547e+19 2.67948547e+19 2.55071281e+19 2.55071281e+19
 2.55071281e+19 2.41933062e+19 2.41933062e+19 2.41933062e+19
 2.29080997e+19 2.29080997e+19 2.29080997e+19 2.16654601e+19
 2.16654601e+19 2.166546

In [15]:
num_species = 3

species_dens_holder = np.zeros((3, int(len(density_profile)/3)))
species_temp_holder = np.zeros((3, int(len(temp_profile)/3)))

i = 0
j = 0
while i < int(len(density_profile)/3):
    for k in range(num_species):
        species_dens_holder[k, i] = density_profile[j]
        species_temp_holder[k, i] = temp_profile[j]
        j = j + 1
    i = i + 1
    


print(species_dens_holder[0, :])

print('Saving to file....')
np.save('../../../PetraM/genray_density', species_dens_holder[0,:])
np.savetxt('../../../PetraM/genray_density.txt', species_dens_holder[0,:])
# print(species_temp_holder[1, :])
# print(species_temp_holder[2, :])

with open(f'genray.in') as file:
    lines = file.readlines()

first_den_scale = True
first_tem_scale = True
for line in lines:
    if 'ndens' in line:
        ndens = int(line.split('=')[1].split('!')[0].strip())
        print(ndens)
    if 'den_scale(3)' in line and first_den_scale == True:
        den_scale_3 = float(line.split('=')[1].split('!')[0].strip())
        first_den_scale = False
    if 'temp_scale(3)' in line and first_tem_scale == True:
         tem_scale_3 = float(line.split('=')[1].split('!')[0].strip())
         first_tem_scale = False       

genray_rho = np.linspace(0, 1, ndens)
#print(genray_rho)
print(den_scale_3)

[4.48613453e+19 4.46052391e+19 4.39110362e+19 4.27147693e+19
 4.10693018e+19 3.95449586e+19 3.81727751e+19 3.69408239e+19
 3.58330029e+19 3.48182826e+19 3.38667281e+19 3.29523368e+19
 3.20500938e+19 3.11325085e+19 3.01717716e+19 2.91387608e+19
 2.80132527e+19 2.67948547e+19 2.55071281e+19 2.41933062e+19
 2.29080997e+19 2.16654601e+19 2.03960024e+19 1.89023392e+19
 1.63783758e+19 4.87435790e+18]
Saving to file....
26
0.1


In [14]:
rr = np.load('/home/vandelij/Desktop/HFS_HHFW_antenna/PetraM/genray_density.npy')

rr

array([4.48613453e+19, 4.46052391e+19, 4.39110362e+19, 4.27147693e+19,
       4.10693018e+19, 3.95449586e+19, 3.81727751e+19, 3.69408239e+19,
       3.58330029e+19, 3.48182826e+19, 3.38667281e+19, 3.29523368e+19,
       3.20500938e+19, 3.11325085e+19, 3.01717716e+19, 2.91387608e+19,
       2.80132527e+19, 2.67948547e+19, 2.55071281e+19, 2.41933062e+19,
       2.29080997e+19, 2.16654601e+19, 2.03960024e+19, 1.89023392e+19,
       1.63783758e+19, 4.87435790e+18])